
Se Implementa un Raspador Web, utilizando una Clase de Python, que extraiga
el título, el cuerpo y la fecha de publicación de un artículo de noticias desde el
sitio web elegido.

Se utilizará el sitio web BBC news


https://www.bbc.com/mundo/noticias-internacional-58490346

In [1]:
webBBC = 'https://www.bbc.com/mundo/noticias-internacional-58490346'

In [2]:
import requests
from bs4 import BeautifulSoup
import datetime 

class Content:
    def __init__(self, url, title, body, date):
        self.url = url
        self.title = title
        self.body = body
        self.date = date
        
def getPage(url):            
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')

def scrapeBBC(url):
    bs = getPage(url)
    title = bs.find("h1").text
    lines = bs.find_all("div", {"class":"bbc-19j92fr e57qer20"})
    date ='Artículo publicado hace '+bs.find('time',{'class':'bbc-14xtggo e4zesg50'}).text
    now = datetime.datetime.now()
    date = now - datetime.timedelta(hours=int(bs.find('time',{'class':'bbc-14xtggo e4zesg50'}).text[0]))
    body = '\n'.join([line.text for line in lines])
    return Content(url, title, body, date)


In [3]:
content=scrapeBBC(webBBC)
print('Title: {}\n'.format(content.title))
print('Date: {}\n'.format(content.date))
print('URL: {}\n'.format(content.url))
print(content.body)

Title: "Fue la noche más triste de todas": las 12 frenéticas horas en las que Kabul cayó en manos del Talibán

Date: 2021-09-24 11:46:04.112356

URL: https://www.bbc.com/mundo/noticias-internacional-58490346

El Talibán anunció un nuevo gobierno en Kabul, 20 años después de que fuera expulsado del poder.
Para una generación que creció con educación, inversión internacional y esperanza en un futuro democrático aquello debió sonar increíble.
Pero ¿cómo cayó la administración anterior tan rápidamente? 
En solo 10 días los talibanes pasaron de tomar el control de una primera ciudad importante a llegar a las puertas de la capital de Afganistán.
Se suponía que la capital sería diferente. La mayoría de los observadores creían que Kabul se mantendría hasta que se pudiera llegar a un acuerdo negociado. 
El domingo 15 de agosto todo cambió. En cuestión de horas, el presidente y sus principales funcionarios habían huido. 
Los miembros que quedaban del ejército afgano y las fuerzas policiales se q

Por fortuna o por desgracia, el sitio de BBC no contiene un buscador dentro de la página web. Es por este motivo que utilizaremos la pagina  'Educational Massachusetts Institute of Technology' para tener consistencia.

https://news.mit.edu/


In [4]:
webMIT='https://news.mit.edu/'

In [5]:
import requests
from bs4 import BeautifulSoup
import datetime 

class Content:
    def __init__(self, url, title, body, date):
        self.url = url
        self.title = title
        self.body = body
        self.date = date
        
def getPage(url):            
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')

def scrapeMIT(url):
    bs = getPage(url)
    title = bs.find("h1").text
    lines = bs.find_all("div", {"class":'paragraph paragraph--type--content-block-text paragraph--view-mode--default'})
    date =bs.find('time').text
    body = '\n'.join([line.text for line in lines])
    return Content(url, title, body, date)

In [6]:
content=scrapeMIT('https://news.mit.edu/2021/pancreatic-organoids-cancer-0913')
print('Title: {}\n'.format(content.title))
print('Date: {}\n'.format(content.date))
print(content.body)
print('URL: {}\n'.format(content.url))


Title: Engineers grow pancreatic “organoids” that mimic the real thing 

Date: September 13, 2021


MIT engineers, in collaboration with scientists at Cancer Research UK Manchester Institute, have developed a new way to grow tiny replicas of the pancreas, using either healthy or cancerous pancreatic cells. Their new models could help researchers develop and test potential drugs for pancreatic cancer, which is currently one of the most difficult types of cancer to treat.

Using a specialized gel that mimics the extracellular environment surrounding the pancreas, the researchers were able to grow pancreatic “organoids,” allowing them to study the important interactions between pancreatic tumors and their environment. Unlike some of the gels now used to grow tissue, the new MIT gel is completely synthetic, easy to assemble and can be produced with a consistent composition every time.

“The issue of reproducibility is a major one,” says Linda Griffith, the School of Engineering Professor o

Se expande el raspador anterior y se implementa un método (para la clase que
define al raspador) que identifica la barra de búsqueda del sitio web elegido,
agrega un tópico de interés, y si encuentra artículos de noticias asociado al
tópico, extrae el título, cuerpo y fecha de publicación de todos los artículos
encontrados en la primera página de resultados, asociados al tópico de interés.


In [7]:
import requests
from bs4 import BeautifulSoup

class Content:
    def __init__(self, topic, url, title, body, date):
        self.topic = topic
        self.title = title
        self.body = body
        self.url = url
        self.date = date
        
    def print(self):
        print(" New article found for topic : {}".format(self.topic))
        print('\n Date of publication : {}'.format(self.date))
        print("\n TITLE : {}".format(self.title))
        print("\n BODY :{}".format(self.body))
        print(" URL : {}".format(self.url))
        
class Website:
    def __init__(self, name, url, searchUrl, resultListing, resultUrl, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl   #define a donde debe ir para obtener resultados de busqueda si agrega el tema que esta buscando
        self.resultListing = resultListing   #define el "box-cuadro" que contiene informacion sobre cada resultado
        self.resultUrl = resultUrl   #define la etiqueta dentro de este box que le dara la URL exacta para el resultado
        self.absoluteUrl=absoluteUrl   #es un valor booleano que le dice si estos resultados de busqueda son URL absolutas o relativas
        self.titleTag = titleTag
        self.bodyTag = bodyTag

class Crawler:
    
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        
        return BeautifulSoup(req.text, 'html.parser')
    
    def safeGet(self, pageObj, selector):
        childObj = pageObj.select(selector)
        
        if childObj is not None and len(childObj) > 0:
            return childObj[0].get_text()
        return ''
        
    
    def search(self, topic, site):
        bs = self.getPage(site.searchUrl + topic)
        searchResults = bs.select(site.resultListing)
        
        #agregamos una salida para cuando no tengamos coincidencias en la búsqueda
        
        if searchResults is None or searchResults==[]:
            print('No se encontraron coincidencias acerca de \'{}\'.'.format(topic))
        
        for result in searchResults:
            url = result.select(site.resultUrl)[0].attrs["href"]
            
            if(site.absoluteUrl):
                bs = self.getPage(url)
            else:
                url = site.url + url
                bs = self.getPage(url)
                
            if bs is None:
                print("Ocurrió algo no deseado. Revisar!!")
                return
            
            title = self.safeGet(bs, site.titleTag)
            body = self.safeGet(bs, site.bodyTag)
            date= bs.find('time').text
            
            if title != '' and body != '':
                content = Content(topic, url, title, body, date)
                content.print()
                # agregarmos un separador para cada artículo
                print('\n_____________________________________________________________________________________________________________________________\n')
    
    
        
             
crawler = Crawler()

In [21]:
website = Website('MIT',
                  'https://news.mit.edu/',
                  'https://news.mit.edu/search?keyword=',
                  'div.search-result-item--cover-image',
                  'a',
                  False,
                  'h1',
                  'div.news-article--content--body'
                 )

crawler.search('Modern theroy', website)

No se encontraron coincidencias acerca de 'Modern teory of measure'.


In [10]:
crawler.search('Deep Learning', website)

 New article found for topic : Deep Learning

 Date of publication : September 23, 2021

 TITLE : Toward a smarter electronic health record 

 BODY :


Electronic health records have been widely adopted with the hope they would save time and improve the quality of patient care. But due to fragmented interfaces and tedious data entry procedures, physicians often spend more time navigating these systems than they do interacting with patients.

Researchers at MIT and the Beth Israel Deaconess Medical Center are combining machine learning and human-computer interaction to create a better electronic health record (EHR). They developed MedKnowts, a system that unifies the processes of looking up medical records and documenting patient information into a single, interactive interface.

Driven by artificial intelligence, this “smart” EHR automatically displays customized, patient-specific medical records when a clinician needs them. MedKnowts also provides autocomplete for clinical terms and a

Se implementa una función o un método para la clase que define al Raspador para
guardar los datos extraídos de forma ordenada y en un archivo formato CSV.

In [22]:
import csv

class new_Crawler:
    
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        
        return BeautifulSoup(req.text, 'html.parser')
    
    def safeGet(self, pageObj, selector):
        childObj = pageObj.select(selector)
        
        if childObj is not None and len(childObj) > 0:
            return childObj[0].get_text()
        return ''
        
    
    def search(self, topic, site):
        bs = self.getPage(site.searchUrl + topic)
        searchResults = bs.select(site.resultListing)
        
        if searchResults is None or searchResults==[]:
            print('No se encontraron coincidencias de \'{}\'.'.format(topic))
        
        ##Creamos lista donde se almacenarán todos los Articulos
        Articles=[]
        
        for result in searchResults:
            url = result.select(site.resultUrl)[0].attrs["href"]
            
            if(site.absoluteUrl):
                bs = self.getPage(url)
            else:
                url = site.url + url
                bs = self.getPage(url)
                
            if bs is None:
                print("Ocurrió algo no deseado. Revisar!!")
                return
            
            title = self.safeGet(bs, site.titleTag)
            body = self.safeGet(bs, site.bodyTag)
            date= bs.find('time').text
            
            if title != '' and body != '':
                content = Content(topic, url, title, body, date)
                ##Creamos una lista que contenga el titulo, fecha y cuerpo del artículo y la almacenamos en Articles
                Articles.append([content.title,content.date,content.body])
        
        #retornamos un array donde cada elemento consistirá en un bs.
        return Articles

    ##Mediante newCrawler.search() podemos guardar la información 
    def save_csv(self, Articles , rute , archive_name):
        
        with open(rute+'\\'+ archive_name+'','w',encoding='utf-8', newline='') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow(('Title','Date of publication','Body'))
            
            for article in Articles:
                writer.writerow(article)
            
new_Crawler=new_Crawler()        

In [23]:
website = Website('MIT',
                  'https://news.mit.edu/',
                  'https://news.mit.edu/search?keyword=',
                  'div.search-result-item--cover-image',
                  'a',
                  False,
                  'h1',
                  'div.news-article--content--body'
                 )

In [24]:
rute = r'CSV'

label1 = 'Obama'
label2 = 'Machine Learning'

In [25]:
Articles1 = new_Crawler.search(label1, website)
new_Crawler.save_csv(Articles1, rute ,label1+'_MIT')


In [15]:
Articles2 = new_Crawler.search(label2, website)
new_Crawler.save_csv(Articles2, rute, label2+'_MIT')


In [26]:
import pandas as pd
a1=pd.read_csv(r'CSV\Obama_MIT')

In [27]:
a1

,Title,Date of publication,Body
0,"Sylvester James Gates Jr. ’73, PhD ’77 recount...","September 23, 2021","\n\n\nSylvester James Gates Jr. ’73, PhD ’77 h..."
1,"Reflecting on September 11, 20 years later","September 9, 2021",\n\n\nThe 20th anniversary of the attacks of S...
2,Media Advisory — MIT researchers: AI policy ne...,"May 4, 2021","\n\n\nOn Thursday, May 6 and Friday, May 7, th..."
3,"In Compton Lecture, Kwame Anthony Appiah analy...","April 26, 2021",\n\n\nPhilosopher Kwame Anthony Appiah offered...
4,MIT launches new data privacy-focused initiative,"April 20, 2021",\n\n\nStrategic use of data is vital for progr...
5,Retrofitting MIT’s deep learning “boot camp” f...,"March 4, 2021",\n\n\nDeep learning is advancing at lightning ...
6,An aggressive market-driven model for US fusio...,"February 24, 2021",\n\n\nElectricity generated by fusion power pl...
7,Q&A: Ceasar McDowell on better public conversa...,"February 24, 2021","\n\n\nLast year leaders in Poughkeepsie, New Y..."
8,Ijeoma Oluo: “What we are fighting for is Blac...,"February 11, 2021",\n\n\nIn a moving and deeply personal address ...
9,Foreign policy advice: Don’t look back,"January 29, 2021",\n\n\nPresident Joe Biden’s administration rep...


In [18]:
new_Crawler.search('Theorem',website)
new_Crawler.save_csv(Articles1, rute ,'Theorem_MIT')

In [19]:
a1=pd.read_csv(r'CSV\Theorem_MIT')

In [20]:
a1

,Title,Date of publication,Body
0,"Sylvester James Gates Jr. ’73, PhD ’77 recount...","September 23, 2021","\n\n\nSylvester James Gates Jr. ’73, PhD ’77 h..."
1,"Reflecting on September 11, 20 years later","September 9, 2021",\n\n\nThe 20th anniversary of the attacks of S...
2,Media Advisory — MIT researchers: AI policy ne...,"May 4, 2021","\n\n\nOn Thursday, May 6 and Friday, May 7, th..."
3,"In Compton Lecture, Kwame Anthony Appiah analy...","April 26, 2021",\n\n\nPhilosopher Kwame Anthony Appiah offered...
4,MIT launches new data privacy-focused initiative,"April 20, 2021",\n\n\nStrategic use of data is vital for progr...
5,Retrofitting MIT’s deep learning “boot camp” f...,"March 4, 2021",\n\n\nDeep learning is advancing at lightning ...
6,An aggressive market-driven model for US fusio...,"February 24, 2021",\n\n\nElectricity generated by fusion power pl...
7,Q&A: Ceasar McDowell on better public conversa...,"February 24, 2021","\n\n\nLast year leaders in Poughkeepsie, New Y..."
8,Ijeoma Oluo: “What we are fighting for is Blac...,"February 11, 2021",\n\n\nIn a moving and deeply personal address ...
9,Foreign policy advice: Don’t look back,"January 29, 2021",\n\n\nPresident Joe Biden’s administration rep...
